In [5]:
import os
import logging
from pathlib import Path

import numpy as np
import pandas as pd
import librosa
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm
import sys
from joblib import Parallel, delayed
logging.basicConfig(level=logging.ERROR)


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
# 
class CFG:
    def __init__(self, mode="train", kaggle_notebook=False, debug=False):
        assert mode in ["train", "inference"], "mode must be 'train' or 'inference'"
        self.mode = mode
        self.KAGGLE_NOTEBOOK = kaggle_notebook
        self.debug = debug

        # ===== Path Settings =====
        if self.KAGGLE_NOTEBOOK:
            self.OUTPUT_DIR = ''
            self.train_datadir = '/kaggle/input/birdclef-2025/train_audio'
            self.train_csv = '/kaggle/input/birdclef-2025/train.csv'
            self.test_soundscapes = '/kaggle/input/birdclef-2025/test_soundscapes'
            self.submission_csv = '/kaggle/input/birdclef-2025/sample_submission.csv'
            self.taxonomy_csv = '/kaggle/input/birdclef-2025/taxonomy.csv'
            self.spectrogram_npy = '/kaggle/input/birdclef25-mel-spectrograms/birdclef2025_melspec_5sec_256_256.npy'
            
            # kaggle notebookならここを変更
            self.model_path = "/kaggle/input/birdclef-2025-baseline-fold0-0404"
            
            self.device = "cpu"
            self.batch_size = 8
            self.n_jobs = 3
            
        else:
            self.OUTPUT_DIR = '../data/result/'
            self.train_datadir = '../data/raw/train_audio/'
            self.train_csv = '../data/raw/train.csv'
            self.test_soundscapes = '../data/raw/test_soundscapes/'
            self.submission_csv = '../data/raw/sample_submission.csv'
            self.taxonomy_csv = '../data/raw/taxonomy.csv'
            self.spectrogram_npy = '../data/processed/mel-spec_0329/birdclef2025_melspec_5sec_256_256.npy'
            self.MODELS_DIR = "../models/"
            
            # ローカルならここを変更
            self.model_path =  "../models/fold0_cleaned_10sec0423_vino/"
            
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
            self.batch_size = 32
            self.n_jobs = 3

        # ===== Model Settings =====
        self.model_name = 'efficientnet_b0'
        self.pretrained = True if mode == "train" else False
        self.in_channels = 1

        # ===== Audio Settings =====
        self.FS = 32000
        self.TARGET_SHAPE = (148, 640)
        self.N_FFT = 1024
        self.HOP_LENGTH = 64
        self.N_MELS = 148
        self.FMIN = 20
        self.FMAX = 16000
        
        self.seed = 42
        
        # smoothingの係数
        self.smooth_center_weight = 0.6
        self.smooth_neighbor_weight = 0.2
        
        # ===== Inference Mode =====
        if mode == "inference":
            self.use_tta = False
            self.tta_count = 3
            self.threshold = 0.5

            self.use_specific_folds = False
            self.folds = [0, 1, 2, 3, 4]  # Used only if use_specific_folds is True

            self.debug_count = 3
            self.INFER_WINDOW_SEC = 10  # 推論時に使う1ウィンドウの長さ（秒）
            self.INFER_HOP_SEC = 5      # 推論時のスライド幅（秒）
            
            
    def update_debug_settings(self):
        if self.debug:
            self.epochs = 2
            self.selected_folds = [0]

In [7]:
cfg = CFG(mode='inference', kaggle_notebook=False)

if cfg.KAGGLE_NOTEBOOK:
    !pip install -U openvino-telemetry  --no-index --find-links /kaggle/input/pip-hub
    !pip install -U openvino  --no-index --find-links /kaggle/input/pip-hub
    sys.path.append("/kaggle/input/birdclef-2025-libs/")
    
from openvino.runtime import Core
from module import models_lib, utils_lib, preprocess_lib, inference_lib

# Set seed
utils_lib.set_seed(cfg.seed)

In [8]:
print(f"Using device: {cfg.device}")
print(f"Loading taxonomy data...")
taxonomy_df = pd.read_csv(cfg.taxonomy_csv)
species_ids = taxonomy_df['primary_label'].tolist()
num_classes = len(species_ids)
print(f"Number of classes: {num_classes}")

Using device: cuda
Loading taxonomy data...
Number of classes: 206


In [ ]:
# mel変換
def process_audio_file(audio_path, cfg):
    """
    row_id = 5〜60（12個）に対する10秒ウィンドウの start/end を明示的に定義して処理。
    """
    dataset = []
    soundscape_id = Path(audio_path).stem
    audio_data, _ = librosa.load(audio_path, sr=cfg.FS)
    fs = cfg.FS

    # === 明示的なウィンドウ定義 ===　←頭いい
    segments = [
        (5, 0.0, 10.0),
        (10, 2.5, 12.5),
        (15, 7.5, 17.5),
        (20, 12.5, 22.5),
        (25, 17.5, 27.5),
        (30, 22.5, 32.5),
        (35, 27.5, 37.5),
        (40, 32.5, 42.5),
        (45, 37.5, 47.5),
        (50, 42.5, 52.5),
        (55, 47.5, 57.5),
        (60, 50.0, 60.0),
    ]

    for row_id_time, start_sec, end_sec in segments:
        start_idx = int(start_sec * fs)
        end_idx = int(end_sec * fs)
        segment = audio_data[start_idx:end_idx]

        mel_spec = preprocess_lib.process_audio_segment(segment, cfg)
        row_id = f"{soundscape_id}_{row_id_time}"
        dataset.append({
            "row_id": row_id,
            "mel_spec": mel_spec
        })

    return dataset


# 並列化してmelspecを生成
def generate_melspec_dataset(cfg):
    test_dir = Path(cfg.test_soundscapes)
    if not test_dir.exists():
        print(f"Test directory {test_dir} does not exist.")
        return []

    test_files = list(test_dir.glob('*.ogg'))
    if len(test_files) == 0:
        print("No test audio files found.")
        return []

    if cfg.debug:
        print(f"Debug mode enabled, using only {cfg.debug_count} files")
        test_files = test_files[:cfg.debug_count]

    results = Parallel(n_jobs=cfg.n_jobs)(
        delayed(process_audio_file)(path, cfg) for path in tqdm(test_files, desc="Parallel melspec gen")
    )
    dataset = [item for sublist in results for item in sublist]
    return dataset

In [10]:

# openvinoモデルの読み込み
def load_openvino_models(vino_dir, cfg):
    models = []
    vino_dir = Path(vino_dir)

    if cfg.use_specific_folds:
        fold_ids = cfg.folds
        xml_files = [vino_dir / f"model_fold{f}.xml" for f in fold_ids]
    else:
        xml_files = sorted(vino_dir.glob("model_fold*.xml"))

    for xml_path in xml_files:
        bin_path = xml_path.with_suffix(".bin")

        if not xml_path.exists() or not bin_path.exists():
            print(f"⚠️ Warning: Missing files for {xml_path.stem}")
            continue

        core = Core()
        model_ir = core.read_model(xml_path)
        compiled_model = core.compile_model(model_ir, device_name="CPU")
        models.append(compiled_model)

        # 🔍 モデルのファイル名（fold情報）をログに出す
        print(f"✅ Loaded model: {xml_path.name}")

    print(f"🎉 Total {len(models)} OpenVINO model(s) loaded from {vino_dir}")
    return models

# openvinoモデルによる推論
def run_inference_openvino(dataset, models_ir, cfg, species_ids):
    row_ids = []
    all_preds = []

    for i in range(0, len(dataset), cfg.batch_size):
        batch = dataset[i:i+cfg.batch_size]

        mel_list = [item["mel_spec"] for item in batch]
        input_tensor = np.stack(mel_list).astype(np.float32)  # (B, H, W)
        input_tensor = np.expand_dims(input_tensor, axis=1)  # (B, 1, H, W)

        preds_per_model = []
        for model in models_ir:
            input_layer = model.input(0)
            output_layer = model.output(0)
            result = model([input_tensor])[output_layer]
            probs = 1 / (1 + np.exp(-result))  # sigmoid
            preds_per_model.append(probs)

        avg_preds = np.mean(preds_per_model, axis=0)
        all_preds.append(avg_preds)
        row_ids.extend([item["row_id"] for item in batch])

    predictions = np.concatenate(all_preds, axis=0)
    return row_ids, predictions

In [11]:
# 予測値をsmoothingする．
def smooth_submission_df(submission_df, cfg):
    """
    Smooth predictions in a submission DataFrame and return the modified DataFrame.

    Each row's prediction is averaged with its neighbors using weights from cfg.

    :param submission_df: DataFrame containing submission data (row_id + prediction columns)
    :param cfg: Configuration object containing smoothing weights
    :return: Smoothed submission DataFrame
    """
    print("Smoothing submission predictions...")
    sub = submission_df.copy()
    cols = sub.columns[1:]
    groups = sub['row_id'].astype(str).str.rsplit('_', n=1).str[0].values
    unique_groups = np.unique(groups)

    for group in unique_groups:
        idx = np.where(groups == group)[0]
        sub_group = sub.iloc[idx].copy()
        predictions = sub_group[cols].values
        new_predictions = predictions.copy()

        if predictions.shape[0] > 1:
            w_c = cfg.smooth_center_weight
            w_n = cfg.smooth_neighbor_weight
            new_predictions[0] = predictions[0] * (1 - w_n) + predictions[1] * w_n
            new_predictions[-1] = predictions[-1] * (1 - w_n) + predictions[-2] * w_n
            for i in range(1, predictions.shape[0] - 1):
                new_predictions[i] = (
                    predictions[i - 1] * w_n +
                    predictions[i] * w_c +
                    predictions[i + 1] * w_n
                )
        sub.iloc[idx, 1:] = new_predictions

    return sub

In [12]:
print("Generating dataset...")
dataset = generate_melspec_dataset(cfg)

print("Loading OpenVINO models...")
vino_dir = Path(cfg.model_path).with_name(Path(cfg.model_path).name)
models_ir = load_openvino_models(vino_dir, cfg)

if not models_ir:
    raise RuntimeError("No OpenVINO models found.")

print("Running OpenVINO inference...")
if len(dataset) > 0:
    row_ids, predictions = run_inference_openvino(dataset, models_ir, cfg, species_ids)
else:
    print("No test data available, generating empty submission.")
    row_ids = []
    predictions = []

# smoothing前の予測値を保存
submission_df = utils_lib.create_submission(row_ids, predictions, species_ids, cfg)
submission_path = os.path.join(cfg.OUTPUT_DIR, 'submission_before_smoothing.csv')
submission_df.to_csv(submission_path, index=False)

print(f"Submission saved to {submission_path}")


Generating dataset...


Parallel melspec gen:   0%|          | 0/71 [00:00<?, ?it/s]

Loading OpenVINO models...
✅ Loaded model: model_fold0.xml
🎉 Total 1 OpenVINO model(s) loaded from ../models/fold0_cleaned_10sec0423_vino
Running OpenVINO inference...
Creating submission dataframe...
Submission saved to ../data/result/submission_before_smoothing.csv


In [13]:
# smoothingして再度保存．
smoothed_df = smooth_submission_df(submission_df, cfg)
smoothed_submission_path = os.path.join(cfg.OUTPUT_DIR, 'submission.csv')
smoothed_df.to_csv(smoothed_submission_path, index=False)
print(f"Smoothed submission saved to {smoothed_submission_path}")

Smoothing submission predictions...
Smoothed submission saved to ../data/result/submission.csv


In [14]:
# 提出用ファイルを読み込む
submission = pd.read_csv(os.path.join(cfg.OUTPUT_DIR, 'submission.csv'))
submission.head(12)

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H02_20230420_074000_5,0.000573,0.000131,0.000629,0.000010,0.000721,0.000024,0.002406,0.000667,0.000885,...,0.059790,0.012822,0.000100,0.002834,0.000179,0.006928,0.001649,0.031621,0.018129,0.000020
1,H02_20230420_074000_10,0.000342,0.000071,0.000446,0.000009,0.000520,0.000011,0.001486,0.000465,0.000739,...,0.030302,0.006982,0.000052,0.002215,0.000105,0.005095,0.000766,0.016129,0.009563,0.000016
2,H02_20230420_074000_15,0.000268,0.000044,0.000417,0.000013,0.000403,0.000006,0.001368,0.000416,0.000927,...,0.012670,0.003747,0.000049,0.002466,0.000130,0.005473,0.000451,0.006312,0.004056,0.000018
3,H02_20230420_074000_20,0.000362,0.000076,0.000653,0.000019,0.000484,0.000009,0.001895,0.000563,0.001398,...,0.013731,0.007850,0.000098,0.002880,0.000262,0.007796,0.001022,0.006120,0.004075,0.000025
4,H02_20230420_074000_25,0.000371,0.000111,0.000813,0.000010,0.000529,0.000009,0.001739,0.000637,0.001037,...,0.017475,0.012832,0.000092,0.002184,0.000233,0.007327,0.001631,0.013125,0.005920,0.000023
5,H02_20230420_074000_30,0.000164,0.000050,0.000324,0.000005,0.000271,0.000004,0.000789,0.000330,0.000337,...,0.011664,0.006680,0.000039,0.001257,0.000085,0.003024,0.000728,0.016453,0.003981,0.000015
6,H02_20230420_074000_35,0.000121,0.000033,0.000201,0.000006,0.000220,0.000003,0.000692,0.000269,0.000241,...,0.007500,0.006499,0.000044,0.000959,0.000102,0.004033,0.000326,0.008518,0.003260,0.000013
7,H02_20230420_074000_40,0.000151,0.000038,0.000228,0.000007,0.000173,0.000003,0.000696,0.000242,0.000284,...,0.006962,0.010681,0.000084,0.001355,0.000164,0.007299,0.000571,0.004175,0.006759,0.000018
8,H02_20230420_074000_45,0.000295,0.000067,0.000444,0.000006,0.000299,0.000004,0.001393,0.000409,0.000402,...,0.011733,0.012788,0.000093,0.001963,0.000144,0.005643,0.001070,0.007651,0.008391,0.000023
9,H02_20230420_074000_50,0.000369,0.000098,0.000675,0.000007,0.000444,0.000007,0.001835,0.000560,0.000504,...,0.015689,0.013871,0.000076,0.002106,0.000168,0.005856,0.001626,0.010444,0.008657,0.000037


In [15]:
submission = pd.read_csv(os.path.join(cfg.OUTPUT_DIR, 'submission_before_smoothing.csv'))
submission.head()

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H02_20230420_074000_5,0.000646,0.000149,0.000688,0.000011,0.000779,0.000028,0.002715,0.000732,0.000940,...,0.068537,0.014504,0.000116,0.003075,0.000203,0.007577,0.001927,0.036220,0.020646,0.000021
1,H02_20230420_074000_10,0.000278,0.000058,0.000392,0.000007,0.000488,0.000008,0.001170,0.000404,0.000667,...,0.024798,0.006098,0.000038,0.001868,0.000080,0.004332,0.000536,0.013223,0.008062,0.000014
2,H02_20230420_074000_15,0.000231,0.000033,0.000368,0.000011,0.000357,0.000004,0.001207,0.000380,0.000755,...,0.008577,0.002116,0.000031,0.002397,0.000085,0.004905,0.000296,0.004755,0.002981,0.000017
3,H02_20230420_074000_20,0.000368,0.000062,0.000588,0.000024,0.000458,0.000010,0.002051,0.000536,0.001701,...,0.012821,0.006290,0.000116,0.003273,0.000315,0.008316,0.000830,0.004070,0.003273,0.000028
4,H02_20230420_074000_25,0.000473,0.000158,0.001134,0.000008,0.000688,0.000010,0.002116,0.000830,0.001134,...,0.021615,0.018264,0.000109,0.002183,0.000278,0.009126,0.002323,0.013637,0.007577,0.000026


In [16]:
submission

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H02_20230420_074000_5,0.000646,0.000149,0.000688,0.000011,0.000779,0.000028,0.002715,0.000732,0.000940,...,0.068537,0.014504,0.000116,0.003075,0.000203,0.007577,0.001927,0.036220,0.020646,0.000021
1,H02_20230420_074000_10,0.000278,0.000058,0.000392,0.000007,0.000488,0.000008,0.001170,0.000404,0.000667,...,0.024798,0.006098,0.000038,0.001868,0.000080,0.004332,0.000536,0.013223,0.008062,0.000014
2,H02_20230420_074000_15,0.000231,0.000033,0.000368,0.000011,0.000357,0.000004,0.001207,0.000380,0.000755,...,0.008577,0.002116,0.000031,0.002397,0.000085,0.004905,0.000296,0.004755,0.002981,0.000017
3,H02_20230420_074000_20,0.000368,0.000062,0.000588,0.000024,0.000458,0.000010,0.002051,0.000536,0.001701,...,0.012821,0.006290,0.000116,0.003273,0.000315,0.008316,0.000830,0.004070,0.003273,0.000028
4,H02_20230420_074000_25,0.000473,0.000158,0.001134,0.000008,0.000688,0.000010,0.002116,0.000830,0.001134,...,0.021615,0.018264,0.000109,0.002183,0.000278,0.009126,0.002323,0.013637,0.007577,0.000026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847,H02_20230502_080500_40,0.000856,0.000231,0.001549,0.000217,0.001648,0.000051,0.000779,0.001245,0.002632,...,0.006098,0.002801,0.000473,0.001501,0.000123,0.033086,0.000278,0.001170,0.014504,0.000169
848,H02_20230502_080500_45,0.000180,0.000021,0.000203,0.000043,0.000444,0.000009,0.000315,0.000180,0.000458,...,0.009708,0.001868,0.000245,0.001325,0.000038,0.006488,0.000131,0.001701,0.074770,0.000096
849,H02_20230502_080500_50,0.000404,0.000090,0.000856,0.000070,0.001134,0.000045,0.001988,0.000646,0.001410,...,0.014504,0.003707,0.000883,0.001134,0.000315,0.015425,0.000278,0.002252,0.027169,0.000217
850,H02_20230502_080500_55,0.000404,0.000102,0.000553,0.000085,0.000883,0.000102,0.000646,0.000570,0.001170,...,0.032101,0.003483,0.000335,0.001455,0.000180,0.020964,0.000158,0.003273,0.021948,0.000261


In [40]:
print("✅ Shape:", submission.shape)
print("✅ Columns:", submission.columns.tolist())
print("✅ Dtypes:\n", submission.dtypes)
print("✅ Nulls:\n", submission.isna().sum().sum())

✅ Shape: (852, 207)
✅ Columns: ['row_id', '1139490', '1192948', '1194042', '126247', '1346504', '134933', '135045', '1462711', '1462737', '1564122', '21038', '21116', '21211', '22333', '22973', '22976', '24272', '24292', '24322', '41663', '41778', '41970', '42007', '42087', '42113', '46010', '47067', '476537', '476538', '48124', '50186', '517119', '523060', '528041', '52884', '548639', '555086', '555142', '566513', '64862', '65336', '65344', '65349', '65373', '65419', '65448', '65547', '65962', '66016', '66531', '66578', '66893', '67082', '67252', '714022', '715170', '787625', '81930', '868458', '963335', 'amakin1', 'amekes', 'ampkin1', 'anhing', 'babwar', 'bafibi1', 'banana', 'baymac', 'bbwduc', 'bicwre1', 'bkcdon', 'bkmtou1', 'blbgra1', 'blbwre1', 'blcant4', 'blchaw1', 'blcjay1', 'blctit1', 'blhpar1', 'blkvul', 'bobfly1', 'bobher1', 'brtpar1', 'bubcur1', 'bubwre1', 'bucmot3', 'bugtan', 'butsal1', 'cargra1', 'cattyr', 'chbant1', 'chfmac1', 'cinbec1', 'cocher1', 'cocwoo1', 'colara1', '